In [2]:
from pathlib import Path
import sys

dir_src = Path.cwd().parent.parent/'src'
sys.path.insert(0, str(dir_src))

In [3]:
import os

from arcgis.gis import GIS
from dotenv import load_dotenv, find_dotenv
from modeling import Country

# load environment variables from .env
load_dotenv(find_dotenv())

True

In [4]:
gis_ent= GIS(os.getenv('ESRI_PORTAL_URL'), username=os.getenv('ESRI_PORTAL_USERNAME'), password=os.getenv('ESRI_PORTAL_PASSWORD'))

gis_ent

GIS @ https://geoai-ent.bd.esri.com/portal/ version:8.2

In [9]:
usa = Country('USA', gis_ent)
#usa = Country('USA', 'local')

usa

<modeling.Country - USA (GIS at https://geoai-ent.bd.esri.com/portal/ logged in as jmccune)>

In [10]:
usa.geography_levels

,geo_name,geo_alias,admin_level,singular_name,plural_name,id
0,block_groups,Block Groups,,Block Group,Block Groups,US.BlockGroups
1,census_tracts,Census Tracts,,Census Tract,Census Tracts,US.Tracts
2,cities_and_towns_places,Cities and Towns (Places),,Place,Places,US.Places
3,zip_codes,ZIP Codes,Admin4,ZIP Code,ZIP Codes,US.ZIP5
4,county_subdivisions,County Subdivisions,,County Subdivision,County Subdivisions,US.CSD
5,counties,Counties,Admin3,County,Counties,US.Counties
6,cbsas,CBSAs,,CBSA,CBSAs,US.CBSA
7,congressional_districts,Congressional Districts,,Congressional District,Congressional Districts,US.CD
8,dmas,DMAs,,DMA,DMAs,US.DMA
9,states,States,Admin2,State,States,US.States


In [11]:
ev = usa.enrich_variables
keep_cols = [c for c in ev.columns if c in ['name', 'alias', 'data_collection', 'description', 'vintage', 'units']]
ev = usa.enrich_variables[keep_cols].copy()
coll_srs = ev.groupby('name')['data_collection'].apply(list)
coll_srs.name = 'collection_tags'
ev = ev.drop_duplicates('name').reset_index(drop=True).drop(columns=['data_collection'])
ev = ev.join(coll_srs, on='name')

ev

,name,alias,description,vintage,units,collection_tags
0,AGE0_CY,2020 Population Age <1,2020 Total Population Age <1 (Esri),2020,count,[1yearincrements]
1,AGE1_CY,2020 Population Age 1,2020 Total Population Age 1 (Esri),2020,count,[1yearincrements]
2,AGE2_CY,2020 Population Age 2,2020 Total Population Age 2 (Esri),2020,count,[1yearincrements]
3,AGE3_CY,2020 Population Age 3,2020 Total Population Age 3 (Esri),2020,count,[1yearincrements]
4,AGE4_CY,2020 Population Age 4,2020 Total Population Age 4 (Esri),2020,count,[1yearincrements]
...,...,...,...,...,...,...
14664,RELRMV1989,2018 RHHs/Moved In: 1989/Before REL (ACS 5-Yr),2018 Renter Households by Year Moved In: 1989/...,2014-2018,count,[yearmovedin]
14665,ACSMEDYRMV,2018 Median Year Householder Moved In (ACS 5-Yr),2018 Median Year Householder Moved into Unit (...,2014-2018,count,[yearmovedin]
14666,MOEMEDYRMV,2018 Median Year Householder Moved In MOE (ACS...,2018 Median Year Householder Moved into Unit M...,2014-2018,count,[yearmovedin]
14667,RELMEDYRMV,2018 Median Year Householder Moved In REL (ACS...,2018 Median Year Householder Moved into Unit R...,2014-2018,count,[yearmovedin]


In [12]:
ev[ev.collection_tags.apply(lambda val: len(val) > 1)]

,name,alias,description,vintage,units,collection_tags
576,POP0_CY,2020 Population Age 0-4,2020 Total Population Age 0-4 (Esri),2020,count,"[5yearincrements, Health, Policy, gender]"
577,POP5_CY,2020 Population Age 5-9,2020 Total Population Age 5-9 (Esri),2020,count,"[5yearincrements, Health, Policy, gender]"
578,POP10_CY,2020 Population Age 10-14,2020 Total Population Age 10-14 (Esri),2020,count,"[5yearincrements, Health, Policy, gender]"
579,POP15_CY,2020 Population Age 15-19,2020 Total Population Age 15-19 (Esri),2020,count,"[5yearincrements, Health, Policy, gender]"
580,POP20_CY,2020 Population Age 20-24,2020 Total Population Age 20-24 (Esri),2020,count,"[5yearincrements, Health, Policy, gender]"
...,...,...,...,...,...,...
14086,THHGRPU4,2020 HHs: Urbanization Group 4,2020 Suburban Periphery Tapestry Urbanization ...,2020,count,"[tapestryhouseholdsNEW, urbanizationgroupsNEW]"
14087,THHGRPU5,2020 HHs: Urbanization Group 5,2020 Semirural Tapestry Urbanization Group U5 ...,2020,count,"[tapestryhouseholdsNEW, urbanizationgroupsNEW]"
14088,THHGRPU6,2020 HHs: Urbanization Group 6,2020 Rural Tapestry Urbanization Group U6 Hous...,2020,count,"[tapestryhouseholdsNEW, urbanizationgroupsNEW]"
14467,ACSBLT2010,2018 HUs/Year Built: 2010-2013 (ACS 5-Yr),2018 Housing Units Built in 2010-2013 (ACS 5-Yr),2014-2018,count,"[unitsinstructure, yearbuilt]"


In [6]:
%%time
df = usa.cbsas.get('seattle').mdl.level(0).get()

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2480 entries, 0 to 2479
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype   
---  ------  --------------  -----   
 0   ID      2480 non-null   object  
 1   NAME    2480 non-null   object  
 2   SHAPE   2480 non-null   geometry
dtypes: geometry(1), object(2)
memory usage: 58.2+ KB
Wall time: 4.74 s


In [7]:
ev = usa.enrich_variables

enrich_variables = ev[
    (ev.data_collection.str.lower().str.contains('key'))  # get the key variables
    & (ev.name.str.endswith('CY'))                     # just current year (2019) variables
].reset_index(drop=True)

enrich_variables.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   name               20 non-null     object
 1   alias              20 non-null     object
 2   data_collection    20 non-null     object
 3   enrich_name        20 non-null     object
 4   enrich_field_name  20 non-null     object
 5   description        20 non-null     object
 6   vintage            20 non-null     object
 7   units              20 non-null     object
dtypes: object(8)
memory usage: 1.4+ KB


In [8]:
enrich_variables.sample(5)

,name,alias,data_collection,enrich_name,enrich_field_name,description,vintage,units
10,RENTER_CY,2020 Renter Occupied HUs,KeyUSFacts,KeyUSFacts.RENTER_CY,KeyUSFacts_RENTER_CY,2020 Renter Occupied Housing Units (Esri),2020,count
1,GQPOP_CY,2020 Group Quarters Population,KeyUSFacts,KeyUSFacts.GQPOP_CY,KeyUSFacts_GQPOP_CY,2020 Group Quarters Population (Esri),2020,count
3,TOTHH_CY,2020 Total Households,KeyUSFacts,KeyUSFacts.TOTHH_CY,KeyUSFacts_TOTHH_CY,2020 Total Households (Esri),2020,count
18,DPOPWRK_CY,2020 Daytime Pop: Workers,KeyUSFacts,KeyUSFacts.DPOPWRK_CY,KeyUSFacts_DPOPWRK_CY,2020 Daytime Population: Workers (Esri),2020,count
11,VACANT_CY,2020 Vacant Housing Units,KeyUSFacts,KeyUSFacts.VACANT_CY,KeyUSFacts_VACANT_CY,2020 Vacant Housing Units (Esri),2020,count


In [9]:
%%time
enrich_df = ent_df.mdl.enrich(enrich_variables)

enrich_df.head()

Wall time: 11.5 s


,ID,NAME,TOTPOP_CY,GQPOP_CY,DIVINDX_CY,TOTHH_CY,AVGHHSZ_CY,MEDHINC_CY,AVGHINC_CY,PCI_CY,...,VACANT_CY,MEDVAL_CY,AVGVAL_CY,POPGRW10CY,HHGRW10CY,FAMGRW10CY,DPOP_CY,DPOPWRK_CY,DPOPRES_CY,SHAPE
0,530330012001,530330012.001,1466,0,55.7,588,2.49,120824,155449,62349,...,27,477551,614634,1.47,1.30,1.72,1524,782,742,"{""rings"": [[[-122.31243049957185, 47.704870499..."
1,530330012002,530330012.002,1233,0,73.5,553,2.23,63251,98980,44392,...,30,833333,800833,0.69,0.53,1.11,1101,573,528,"{""rings"": [[[-122.3178410004258, 47.7076489995..."
2,530330032001,530330032.001,1171,0,26.6,559,2.09,117030,147031,70188,...,3,878788,938628,2.39,2.23,2.86,848,427,421,"{""rings"": [[[-122.38754700019909, 47.675934999..."
3,530330032002,530330032.002,1405,0,46.4,775,1.81,92628,106468,58728,...,34,699728,838340,2.01,1.87,2.66,1203,769,434,"{""rings"": [[[-122.38758899913357, 47.671643999..."
4,530330032003,530330032.003,1868,0,40.5,1118,1.67,84004,101288,60621,...,3,498947,582927,5.28,5.11,5.98,1988,1182,806,"{""rings"": [[[-122.38758899913357, 47.669455999..."


In [11]:
mtch_df = ent_usa.get_enrich_variables_dataframe_from_variable_list(enrich_df.columns)

mtch_df[['name', 'description']]

,name,description
0,TOTPOP_CY,2020 Total Population (Esri)
1,GQPOP_CY,2020 Group Quarters Population (Esri)
2,TOTHH_CY,2020 Total Households (Esri)
3,AVGHHSZ_CY,2020 Average Household Size (Esri)
4,AVGHINC_CY,2020 Average Household Income (Esri)
5,DPOP_CY,2020 Total Daytime Population (Esri)
6,DPOPWRK_CY,2020 Daytime Population: Workers (Esri)
7,DPOPRES_CY,2020 Daytime Population: Residents (Esri)
8,MEDHINC_CY,2020 Median Household Income (Esri)
9,TOTHU_CY,2020 Total Housing Units (Esri)


In [12]:
in_batch_df = enrich_df

In [16]:
fs = in_batch_df[in_batch_df.spatial.name].to_frame().spatial.to_featureset()

fs

<FeatureSet> 2480 features